In [47]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow as tf 
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Sequential
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib import image
from PIL import Image
import glob
import os
import PIL


In [37]:
data = image.imread('../dataset/step4_split_spectrograms/dataset_images_test/bark/approval/appl000000000000.png')

In [46]:
# images are normalized from 0-1. 
images = []
for file in glob.iglob(r'../dataset/step4_split_spectrograms/dataset_images_training/approval_all/approval_bark/appl*.png'):
    images.append(image.imread(file))


In [3]:
# scaler = MinMaxScaler() 

# Robust to outliers with sound there could be for higher pitched
# data_scaled = scaler.fit_transform(data)